In [ ]:
import pandas as pd
import time
import rtsvg
rt = rtsvg.RACETrack()
from xwords import XWords, XWordsSolver
from ollama import chat
from ollama import ChatResponse
from pydantic import BaseModel
import copy
import os
_dir_    = '../../../data/crossword_puzzle_screenshots/'
_files_  = os.listdir(_dir_)
entries_file    = None
geometries_file = None
blockers_file   = None
answers_file    = None
for _file_ in _files_:
    if   'entries'    in _file_: entries_file    = _dir_ + _file_
    elif 'geometries' in _file_: geometries_file = _dir_ + _file_
    elif 'blockers'   in _file_: blockers_file   = _dir_ + _file_
    elif 'answers'    in _file_: answers_file    = _dir_ + _file_
xwords = XWords(rt, entries_file, geometries_file, blockers_file, answers_file)
model  = 'gemma3:27b' # 'mistral-small3.1' #'gemma3:27b'
def promptModel(prompt):
    class Guess(BaseModel):
        guess: str
    response: ChatResponse = chat(model=model, messages=[{ 'role': 'user', 'content':  prompt,},], format=Guess.model_json_schema())
    guess = Guess.model_validate_json(response['message']['content'])
    return guess.guess
print(promptModel('What is 1+2*3+5?  Return a single number.')) # force the model to load

In [ ]:
def createPrompt(clue1, clue1_len, clue1_offset, clue2, clue2_len, clue2_offset):
    _prompt_ = '''You are a crossword puzzle solver. Two clues are given below. Each clue corresponds to a word, and the two words intersect at a common character. For each clue, the following information is provided:

Clue – A brief hint or definition.

Answer length – The number of letters in the answer word.

Index of intersection – The index (starting from 0) of the letter in this word that intersects with the other word.

Your task is to find the most likely pair of words that match the clues and intersect at the given positions with the same letter. Prioritize common crossword-style answers and typical word usage.

Input Format:

Clue 1: "<clue text>", Length: <int>, Intersection Index: <int>  
Clue 2: "<clue text>", Length: <int>, Intersection Index: <int>

OutputFormat:

{
  "answer1": "<answer1>",
  "answer2": "<answer2>",
  "letter": "<common letter>",
   "index_in_answer1": <int>,
   "index_in_answer2": <int>
}

Example Input:

Clue 1: "Feline pet", Length: 3, Intersection Index: 1  
Clue 2: "To catch or seize", Length: 4, Intersection Index: 2

Example Output:

{
  "answer1": "CAT",
  "answer2": "GRAB",
  "letter": "A",
  "index_in_answer1": 1,
  "index_in_answer2": 2
}

Input:

Clue 1: "[__CLUE1__]", Length: [__CLUE1_LEN__], Intersection Index: [__CLUE1_I__]
Clue 2: "[__CLUE2__]", Length: [__CLUE2_LEN__], Intersection Index: [__CLUE2_I__]

Output:
'''
    _prompt_ = _prompt_.replace('[__CLUE1__]', clue1)
    _prompt_ = _prompt_.replace('[__CLUE1_LEN__]', str(clue1_len))
    _prompt_ = _prompt_.replace('[__CLUE1_I__]', str(clue1_offset))
    _prompt_ = _prompt_.replace('[__CLUE2__]', clue2)
    _prompt_ = _prompt_.replace('[__CLUE2_LEN__]', str(clue2_len))
    _prompt_ = _prompt_.replace('[__CLUE2_I__]', str(clue2_offset))
    return _prompt_

def promptModel(prompt):
    class Guess(BaseModel):
        answer1: str
        answer2: str
        letter: str
        index_in_answer1: int
        index_in_answer2: int
    response: ChatResponse = chat(model=model, messages=[{ 'role': 'user', 'content':  prompt,},], format=Guess.model_json_schema())
    guess = Guess.model_validate_json(response['message']['content'])
    return guess.answer1.replace(' ',''), guess.answer2.replace(' ','')

for yi in range(xwords.y_tiles):
    for xi in range(xwords.x_tiles):
        _cell_ = xwords.cells[yi][xi]
        if _cell_.isBlocker() == False:
            dy = 0
            while yi+dy > 0 and xwords.cells[yi+dy-1][xi].isBlocker() == False: dy -= 1
            down_clue_num   = xwords.cells[yi+dy][xi].__cluenum__
            down_clue       = xwords.clue(down_clue_num, 'down')
            down_clue_len   = xwords.numberOfLetters(down_clue_num, 'down')
            dx = 0
            while xi+dx > 0 and xwords.cells[yi][xi+dx-1].isBlocker() == False: dx -= 1
            across_clue_num = xwords.cells[yi][xi+dx].__cluenum__
            across_clue     = xwords.clue(across_clue_num, 'across')
            across_clue_len = xwords.numberOfLetters(across_clue_num, 'across')
            _prompt_        = createPrompt(down_clue, down_clue_len, -dy, across_clue, across_clue_len, -dx)
            down_answer, across_answer = promptModel(_prompt_)
            if len(down_answer) > down_clue_len and len(across_answer) == across_clue_len:
                print('.',end='')
                print(f'{down_clue} {down_answer} | {across_clue} {across_answer}')
            else:
                print('!',end='')
